
## M161 first question notebook
    - created venv python environment
    - shloud create requirements for resusability
    - data .csv files to large to upload to github will resolve thaht later
## Data cleaning I
 1.  check types 
 2. check for null values


In [12]:
import pandas as pd
file_path = 'bigdata2025classification/train.csv'

def load_and_process_data(file_path):
    # Load data from a CSV file
    dataTrain = pd.read_csv(file_path)

    print("Data loaded successfully.")
    print("First 5 rows of the dataset:")
    print(dataTrain.head())

    print("\nData summary:")
    print(dataTrain.info())

    # Check for missing values in the dataframe
    print("\nMissing values in each column:")
    print(dataTrain.isnull().sum())
    
    return dataTrain

dataTrain = load_and_process_data(file_path)

# check column data types
def check_column_types(dataTrain):
    print("\nColumn data types:")
    print(dataTrain.dtypes)

check_column_types(dataTrain)


print(f"dataTrain shape: {dataTrain.shape}")


Data loaded successfully.
First 5 rows of the dataset:
       Id                                              Title  \
0  227464  Netflix is coming to cable boxes, and Amazon i...   
1  244074  Pharrell, Iranian President React to Tehran 'H...   
2   60707                    Wildlife service seeks comments   
3   27883  Facebook teams up with Storyful to launch 'FB ...   
4  169596           Caesars plans US$880 mln New York casino   

                                             Content          Label  
0   if you subscribe to one of three rinky-dink (...  Entertainment  
1   pharrell, iranian president react to tehran '...  Entertainment  
2   the u.s. fish and wildlife service has reopen...     Technology  
3   the very nature of social media means it is o...     Technology  
4   caesars plans us$880 mln new york casino jul ...       Business  

Data summary:
<class 'pandas.DataFrame'>
RangeIndex: 111795 entries, 0 to 111794
Data columns (total 4 columns):
 #   Column   Non-Null Cou

In [13]:
# Keep only the first 10000 instances for faster experimentation
dataTrain = dataTrain.iloc[:10000].reset_index(drop=True)
print(f"Subset shape: {dataTrain.shape}")

Subset shape: (10000, 4)


## Data cleaning II continue
3. check for duplicates
***************************
### note
- the data types of all exeprt Id column is "object" in pandas, it works, but could be converted to String for a performance uplift.
*****************************

## Remove duplicates

In [14]:
# Remove duplicates based on 'Title' and 'Content' columns, keeping the first occurrence
dataTrain = dataTrain.drop_duplicates(subset=['Title', 'Content'], keep='first')
print("\nDuplicates based on Title and Content removed. Data shape:", dataTrain.shape)


# Reset the index after removing duplicates
dataTrain = dataTrain.reset_index(drop=True)
print("\nIndex reset. Data shape:", dataTrain.shape)
dataTrain.info()


Duplicates based on Title and Content removed. Data shape: (9995, 4)

Index reset. Data shape: (9995, 4)
<class 'pandas.DataFrame'>
RangeIndex: 9995 entries, 0 to 9994
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Id       9995 non-null   int64
 1   Title    9995 non-null   str  
 2   Content  9995 non-null   str  
 3   Label    9995 non-null   str  
dtypes: int64(1), str(3)
memory usage: 312.5 KB



### **fixed**
*******************

In [15]:
import numpy as np
dataTrain['text_length'] = dataTrain['Content'].apply(len)
dataTrain['word_count'] = dataTrain['Content'].apply(lambda x: len(str(x).split()))
dataTrain['sentence_count'] = dataTrain['Content'].apply(lambda x: len(str(x).split('.')))
dataTrain['avg_word_length'] = dataTrain['Content'].apply(lambda x: np.mean([len(word) for word in str(x).split()]))

print("\n--- Content Statistics ---")
print(dataTrain[['text_length', 'word_count', 'sentence_count', 'avg_word_length']].describe())


--- Content Statistics ---
        text_length    word_count  sentence_count  avg_word_length
count   9995.000000   9995.000000     9995.000000      9995.000000
mean    2553.360580    421.519560       24.513357         5.063809
std     2225.782633    368.240076       26.454342         0.356137
min       17.000000      3.000000        1.000000         3.750000
25%     1300.500000    216.000000       12.000000         4.834382
50%     2016.000000    335.000000       19.000000         5.054496
75%     3132.000000    512.000000       30.000000         5.267284
max    78614.000000  12562.000000     1343.000000        12.540541



### Remove words not in English dictionary

- **probably could change dictionary for better results but it works...**


In [16]:
import re
import nltk
from nltk.corpus import words

# Download the words corpus if not already present
nltk.download('words')
english_words = set(words.words())

def clean_text(text):
    # Split text into words
    word_list = re.findall(r'\b\w+\b', str(text))
    cleaned_words = []
    for word in word_list:
        # Drop any word not in dictionary
        if word.lower() not in english_words:
            continue
        # Drop words with 2+ repeating chars not in dictionary (redundant now, but kept for clarity)
        if re.search(r'(.)\1{1,}', word):
            if word.lower() not in english_words:
                continue
        cleaned_words.append(word)
    return ' '.join(cleaned_words)

# Apply to both columns
dataTrain['Title'] = dataTrain['Title'].apply(clean_text)
dataTrain['Content'] = dataTrain['Content'].apply(clean_text)


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\odys_\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


**NOTE**
- TAKES A LOT OF TIME TO EXECUTE ~7 min
*********************

In [17]:
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Download required NLTK data if not already present
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def clean_text(text):
    # Expand contractions
    text = contractions.fix(text)
    # Convert to lowercase
    text = text.lower()
    # Remove special characters (keep only letters and spaces)
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Tokenize
    words = text.split()
    # Remove stopwords, lemmatize, and stem
    words = [stemmer.stem(lemmatizer.lemmatize(word)) for word in words if word not in stop_words]
    text = ' '.join(words)
    return text

for col in ['Title', 'Content']:
    dataTrain[col] = dataTrain[col].astype(str).apply(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\odys_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\odys_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\odys_\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [18]:
print(dataTrain.head())

       Id                       Title  \
0  227464  come cabl groceri overlord   
1  244074          presid react happi   
2   60707              wildlif servic   
3   27883                      launch   
4  169596           u new york casino   

                                             Content          Label  \
0  subscrib one three dink compar speak cabl abl ...  Entertainment   
1  presid react happi singer presid took twitter ...  Entertainment   
2  fish wildlif servic comment period addit day p...     Technology   
3  natur social medium often sourc real time brea...     Technology   
4  u new york casino latest news top deck world e...       Business   

   text_length  word_count  sentence_count  avg_word_length  
0         1576         264              15         4.965909  
1         1200         192              10         5.250000  
2         2773         416              34         5.665865  
3         1564         254              13         5.157480  
4         2250  

## Starting future extraction (converting text to numbers for ML algorythms to run)





### TF-IDF Vectorization

We will now use TF-IDF vectorization instead of Bag of Words to represent the text data for classification. TF-IDF often improves performance by reducing the impact of common words and highlighting more informative terms.

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine Title and Content if not already done
if 'Combined' not in dataTrain.columns:
    dataTrain['Combined'] = dataTrain['Title'].fillna('') + ' ' + dataTrain['Content'].fillna('')

# Initialize TF-IDF Vectorizer
# You can tune max_features, ngram_range, etc. for further improvement
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=8000)
dataTrain_tfidf = vectorizer.fit_transform(dataTrain['Combined'])

print('TF-IDF matrix shape:', dataTrain_tfidf.shape)
print('Feature names (first 20):', vectorizer.get_feature_names_out()[:20])

TF-IDF matrix shape: (9995, 8000)
Feature names (first 20): ['aa' 'abandon' 'abdomin' 'abil' 'abl' 'abl find' 'abl get' 'abl make'
 'abl see' 'abl take' 'abl use' 'abnorm' 'aboard' 'abort' 'abroad'
 'abruptli' 'absenc' 'absent' 'absolut' 'absorb']


## Logistic Regression Classifier

Now we will train a Logistic Regression classifier using the Bag of Words features extracted above. We will evaluate its performance using accuracy, precision, recall, and F1-score.

### 5-Fold Cross-Validation

We will now use 5-fold cross-validation to evaluate the Logistic Regression classifier, instead of a single train/test split. This provides a more robust estimate of model performance.

### 5-Fold Cross-Validation with TF-IDF

We will now evaluate the Logistic Regression classifier using 5-fold cross-validation on the TF-IDF features.

In [20]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score, cross_val_predict
# from sklearn.metrics import classification_report

# # Assume the target column is named 'Label' (change if needed)
# if 'Label' not in dataTrain.columns:
#     print("ERROR: 'Label' column not found in dataTrain. Please check your dataset.")
# else:
#     X = dataTrain_tfidf
#     y = dataTrain['Label']
#     clf = LogisticRegression(max_iter=1000,class_weight='balanced',random_state=42)

#     # 5-fold cross-validation scores
#     accuracy_scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
#     print("5-Fold Cross-Validation Accuracy Scores:", accuracy_scores)
#     print("Mean Accuracy:", accuracy_scores.mean())
#     print("Std Dev:", accuracy_scores.std())

#     # Cross-validated predictions for full classification report
#     y_pred_cv = cross_val_predict(clf, X, y, cv=5)
#     print("\nClassification Report (5-fold CV, TF-IDF):\n", classification_report(y, y_pred_cv, zero_division=0))

### Logistic Regression with Built-in Cross-Validation (LogisticRegressionCV)

We will now use `LogisticRegressionCV` from scikit-learn, which performs cross-validated logistic regression and automatically tunes the regularization parameter.

In [26]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report

# Assume the target column is named 'Label' (change if needed)
if 'Label' not in dataTrain.columns:
    print("ERROR: 'Label' column not found in dataTrain. Please check your dataset.")
else:
    X = dataTrain_tfidf
    y = dataTrain['Label']
    clf_cv = LogisticRegressionCV(cv=5, max_iter=1000, random_state=42,class_weight='balanced',scoring='accuracy', n_jobs= 6)
    clf_cv.fit(X, y)
    y_pred_cv = clf_cv.predict(X)
    print("Best C values per class:", clf_cv.C_)
    print('\n ***********************')
    

    print("\nClassification Report (LogisticRegressionCV, 5-fold CV):\n", classification_report(y, y_pred_cv, zero_division=0))
    
    print ('\n classification accuracy a=', clf_cv.score(X, y))
    

d:\Github\bigData\venvEnv\Lib\site-packages\sklearn\linear_model\_logistic.py:1780: FutureWarning: The default value for l1_ratios will change from None to (0.0,) in version 1.10. From version 1.10 onwards, only array-like with values in [0, 1] will be allowed, None will be forbidden. To avoid this warning, explicitly set a value, e.g. l1_ratios=(0,).
  warnings.warn(
d:\Github\bigData\venvEnv\Lib\site-packages\sklearn\linear_model\_logistic.py:1823: FutureWarning: The fitted attributes of LogisticRegressionCV will be simplified in scikit-learn 1.10 to remove redundancy. Set`use_legacy_attributes=False` to enable the new behavior now, or set it to `True` to silence this warning during the transition period while keeping the deprecated behavior for the time being. The default value of use_legacy_attributes will change from True to False in scikit-learn 1.10. See the docstring of LogisticRegressionCV for more details.
  warnings.warn(


Best C values per class: [166.81005372 166.81005372 166.81005372 166.81005372]

 ***********************

Classification Report (LogisticRegressionCV, 5-fold CV):
                precision    recall  f1-score   support

     Business       1.00      1.00      1.00      2249
Entertainment       1.00      1.00      1.00      3974
       Health       1.00      1.00      1.00      1085
   Technology       1.00      1.00      1.00      2687

     accuracy                           1.00      9995
    macro avg       1.00      1.00      1.00      9995
 weighted avg       1.00      1.00      1.00      9995


 classification accuracy a= 0.9996998499249625
